# SI 544: Python for Statistical Analysis 
version 201112.1



In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
evals = pd.read_csv('https://raw.githubusercontent.com/umsi-data-science/data/main/evals.csv')
evals.head()

In [ ]:
evals = evals[['score','age','bty_avg']]

In [ ]:
sns.scatterplot(data=evals, x='bty_avg', y='score')

In [ ]:
sns.pairplot(evals)

In [ ]:
# to calculate all possible correlation coefficients in a dataframe
evals.corr()

A heatmap is a really useful visualization technique for large correlation matrices (less so for small correlation matrices:

In [ ]:
sns.heatmap(evals.corr())

We can also use a different type of color palette to highlight the differences:

In [ ]:
sns.heatmap(evals.corr(),cmap=sns.color_palette("BrBG"))

We can also create a more complex visualization that incorporates
histograms of each variable as well as a scatterplot with a
regression line including confidence intervals.

In [ ]:
g = sns.JointGrid(data=evals,x='bty_avg',y='score')
g = g.plot(sns.regplot, sns.histplot)

## Ordinary Least Squares (OLS) Regression

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

statsmodels uses R-Style formulas: y ~ x1 + x2 + x3 + ...

1. y represents the outcome/dependent variable
2. x1, x2, x3, etc represent explanatory/independent variables 

In [ ]:
model0 = smf.ols("score ~ bty_avg", data=evals).fit()
model0.summary()

We can take a look at the residuals:

In [ ]:
model0.resid

And create a histogram:

In [ ]:
sns.histplot(model0.resid)

We can also look at a scatterplot of residuals vs. bty_score

In [ ]:
ax = sns.scatterplot(x=evals['bty_avg'],y=model0.resid)

In [ ]:
model0.params

In [ ]:
model0.params.loc['bty_avg']

In [ ]:
model0.conf_int()

## Bootstrapping the slope

In [44]:
evals_sample = evals.sample(n=len(evals),replace=True)
evals_sample.head()

,score,age,bty_avg,random
323,4.5,52,2.333,4.2
193,3.9,44,6.500,3.7
319,4.5,52,2.333,4.3
232,4.0,49,6.500,4.2
113,4.0,57,4.333,4.5


In [ ]:
nreps = 1000
slopes = []
for i in range(nreps):
    evals_sample = evals.sample(n=len(evals),replace=True)
    model = smf.ols("score ~ bty_avg", data=evals_sample).fit()
    slope = model.params.loc['bty_avg']
    slopes.append(slope)
mean = np.mean(slopes)
sd = np.std(slopes)
print(mean,sd)

In [ ]:
lower_percentile = np.quantile(slopes,0.025)
upper_percentile = np.quantile(slopes,0.975)

In [ ]:
ax = sns.histplot(slopes)
ax.axvline(mean,color='magenta')
ax.axvline(mean-2*sd,color='green')
ax.axvline(mean+2*sd,color='green')
ax.axvline(model0.conf_int().loc['bty_avg'].iloc[0],color='pink')
ax.axvline(model0.conf_int().loc['bty_avg'].iloc[1],color='pink')
ax.axvline(np.quantile(slopes,0.025),color='blue')
ax.axvline(np.quantile(slopes,0.975),color='blue')

## Hypothesis tests about the mean


In [ ]:
evals.head()

In [ ]:
evals_shuffled = evals.copy()
evals_shuffled['score'] = np.random.permutation(evals['score'].values)


In [ ]:
nreps = 1000
slopes = []
for i in range(nreps):
    evals_shuffled['score'] = np.random.permutation(evals_shuffled['score'].values)
#    evals_sample = evals.sample(n=len(evals),replace=True)
    model = smf.ols("score ~ bty_avg", data=evals_shuffled).fit()
    slope = model.params.loc['bty_avg']
    slopes.append(slope)
mean = np.mean(slopes)
sd = np.std(slopes)
print(mean,sd)



In [ ]:
ax = sns.histplot(slopes)

In [ ]:
obs_slope = model0.params.loc['bty_avg']
obs_slope

In [ ]:
ax = sns.histplot(slopes)
ax.axvline(obs_slope,color='red')

In [ ]:
test_statistic = (obs_slope - mean)/sd

In [ ]:
test_statistic

In [ ]:
import scipy.stats as st
st.norm.cdf(test_statistic,len(evals)-1)

In [ ]:
1 - st.norm.cdf(test_statistic)

In [ ]:
print('%0.8f' % (1 - st.norm.cdf(test_statistic)))